## 1. 웹 크롤링 및 데이터 수집

### 1. 곡 제목 및 곡 코드번호 추출과정

In [18]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

h = {"User-Agent": "Mozilla/5.0..."}

res = req.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=494499914', headers = h)

res.text # html

soup = bs(res.text, 'lxml')

# 노래제목
song = soup.select('div.ellipsis.rank01 >span >a')

# 노래가수
singer = soup.select('div.ellipsis.rank02 >a')

# `.button_etc.like` 클래스를 가진 모든 <button> 태그를 선택
buttons = soup.select('button.button_etc.like')

# 첫 번째와 마지막 두 개의 태그를 제외
selected_buttons = buttons[1:-2]

# 남은 태그들에서 `data-song-no` 속성의 값을 추출
song_numbers = [btn['data-song-no'] for btn in selected_buttons]



songList = []
singerList = []
songNumList = []
rankList = []

for i in range(len(song)):
    songList.append(song[i].text)
    singerList.append(singer[i].text)
    songNumList.append(song_numbers[i])
    rankList.append(i+1)

import pandas as pd
dic = {'번호':rankList,'제목': songList, '가수':singerList, '곡코드번호':songNumList}
melon = pd.DataFrame(dic)

melon.set_index('번호', inplace=True)

melon.to_csv('아이유신나는곡들.csv', encoding='utf-8')




### 2. 가사 크롤링 과정

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

melon = pd.read_csv('아이유신나는곡들.csv', index_col='번호')

browser = webdriver.Chrome()  

lyricsList = []

for num in melon['곡코드번호']:
    browser.get(f'https://www.melon.com/song/detail.htm?songId={num}')
    lyric = browser.find_element(By.CLASS_NAME, "lyric")
    lyricsList.append(lyric.text)

browser.quit()

melon["가사"] = lyricsList

melon.to_csv('아이유_곡_가사.csv', encoding='utf-8')


The chromedriver version (115.0.5790.102) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (116.0.5845.110); currently, chromedriver 116.0.5845.96 is recommended for chrome 116.*, so it is advised to delete the driver in PATH and retry


KeyboardInterrupt: 

## 2. 문장 분리
- ##### 크롤링한 가사 데이터를 문장 단위로 나눈다
- ##### 파이썬 문법으로 엔터(\n)를 기준으로 나눈다

In [ ]:
from konlpy.tag import Twitter
import csv #konlpy의 여러 형태소 분석기들과 CSV 파일을 처리하기 위한 csv 모듈

# 가사 데이터가 들어있는 CSV 파일을 읽어옴
with open('아이유_곡_가사(37개).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    lyrics_data = [row[4] for row in reader]

# Twitter 형태소 분석기 초기화
twitter = Twitter()

# 문장 분리 및 형태소 분석
for song_lyrics in lyrics_data:
    sentences = song_lyrics.split('\n')  # 가사를 문장으로 분리

/Users/idohyeong/anaconda3/lib/python3.11/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 3. 형태소 분석

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

from konlpy.tag import Twitter
import csv

# 가사 데이터가 들어있는 CSV 파일을 읽어옴
with open('아이유_곡_가사(37개).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    lyrics_data = [row[4] for row in reader]

# Twitter 형태소 분석기 초기화
twitter = Twitter()

# 문장 분리 및 형태소 분석
for song_lyrics in lyrics_data:
    sentences = song_lyrics.split('\n')  # 가사를 문장으로 분리
    for sentence in sentences:
        # 각 문장을 twitter.pos() 메서드로 형태소 분석하여 형태소와 해당 태그 정보를 추출
        morphemes = twitter.pos(sentence)  # 형태소 분석
        print(morphemes)  # 형태소 분석 결과 출력

[('가사', 'Noun')]
[('I', 'Alpha'), ('’', 'Punctuation'), ('m', 'Alpha'), ('such', 'Alpha'), ('a', 'Alpha'), ('good', 'Alpha'), ('surfer', 'Alpha')]
[('가라앉지', 'Verb'), ('않기', 'Verb')]
[('비틀', 'Noun'), ('비', 'Noun'), ('비틀', 'Noun'), ('거리', 'Noun'), ('다가', 'Noun'), ('풍덩', 'Adverb'), ('uh', 'Alpha')]
[('빠지더라도', 'Verb'), ('구', 'Modifier'), ('명복', 'Noun'), ('따윈', 'Noun'), ('졸업', 'Noun')]
[]
[('I', 'Alpha'), ('’', 'Punctuation'), ('m', 'Alpha'), ('such', 'Alpha'), ('a', 'Alpha'), ('good', 'Alpha'), ('surfer', 'Alpha')]
[('휩쓸리지', 'Adjective'), ('않기', 'Verb')]
[('울렁', 'Noun'), ('우', 'Adverb'), ('울렁', 'Noun'), ('거리', 'Noun'), ('다가', 'Noun'), ('throw', 'Alpha'), ('up', 'Alpha')]
[('게워내더라도', 'Verb'), ('지는', 'Verb'), ('건', 'Noun'), ('난', 'Noun'), ('못', 'Noun'), ('참아', 'Verb')]
[]
[('제일', 'Noun'), ('높은', 'Adjective'), ('파도', 'Noun')]
[('올라', 'Verb'), ('타타', 'Noun'), ('라차', 'Noun'), ('차우', 'Noun'), ('아', 'Josa')]
[('해일', 'Noun'), ('과', 'Josa'), ('함께', 'Adverb'), ('사라질', 'Verb'), ('타이밍', 'Noun')]
[('그건

## 4. 단어 빈도수 추출
- #### 형태소로 분석된 데이터에서 명사 혹은 원하는 품사의 단어를 추출.
- #### Konlpy를 통해 각 품사 별로 빈도수 체크가 가능.
- #### ‘명사’ 기준 빈도수 체크.

In [ ]:
from konlpy.tag import Twitter
from collections import Counter
import csv
import pandas as pd

# 가사 데이터가 들어있는 CSV 파일을 읽어옴
with open('아이유_곡_가사(37개).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    lyrics_data = [row[4] for row in reader]

# Twitter 형태소 분석기 초기화
twitter = Twitter()

# 형태소 타입 선택 (여기서는 명사로 예시)
selected_pos = ['Noun']

# 형태소 분석 및 빈도수 계산
word_freq = Counter()

for song_lyrics in lyrics_data:
    sentences = song_lyrics.split('\n')  # 가사를 문장으로 분리
    for sentence in sentences:
        morphemes = twitter.pos(sentence)  # 형태소 분석
        for word, pos in morphemes:
            if pos in selected_pos:
                word_freq[word] += 1

# 빈도수 기준으로 내림차순 정렬
sorted_word_freq = dict(sorted(word_freq.items(), key=lambda item: item[1], reverse=True))

df = pd.DataFrame(sorted_word_freq.items(), columns=['단어', '빈도수'])

# 상위 빈도수가 높은 단어 50개 출력
top_n = 30
styled_df = df.head(top_n).style\
    .background_gradient(cmap='viridis', subset=['빈도수'])\
    .set_properties(**{'text-align': 'left'})

styled_df

,단어,빈도수
0,나,118
1,너,115
2,내,113
3,사랑,82
4,말,63
5,그,57
6,난,52
7,이,51
8,네,45
9,날,36


## 5. 데이터 시각화(불용어 제외)

In [ ]:
from konlpy.tag import Okt
import csv
import matplotlib.pyplot as plt
from collections import Counter

# 불용어 파일 읽어오기
with open('불용어.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 가사 데이터가 들어있는 CSV 파일을 읽어옴
with open('아이유_곡_가사(37개).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    lyrics_data = [row[4] for row in reader]

# Okt 형태소 분석기 초기화
okt = Okt()

# 불용어를 포함한 형태소 분석 결과 저장할 리스트
all_morphemes = []

# 문장 분리 및 형태소 분석
for song_lyrics in lyrics_data:
    sentences = song_lyrics.split('\n')  # 가사를 문장으로 분리
    for sentence in sentences:
        morphemes = okt.morphs(sentence)  # 형태소 분석
        # 불용어를 제거하고 형태소만 추출
        filtered_morphemes = [morph for morph in morphemes if morph not in stopwords]
        all_morphemes.extend(filtered_morphemes)

# 형태소 빈도수 계산
word_freq = Counter(all_morphemes)
sorted_word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))

# 시각화를 위해 단어와 빈도수 추출
top_words = list(sorted_word_freq.keys())[:30]
freq_counts = list(sorted_word_freq.values())[:30]

# 시각화
plt.figure(figsize=(10, 6))
plt.bar(top_words, freq_counts)
plt.xlabel('word')
plt.ylabel('빈도수')
plt.title('상위 단어 빈도수')
plt.xticks(rotation=45)
plt.rc('font', family='NanumGothic')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding, LSTM, Dense

# 데이터 로드
data = pd.read_csv('아이유_곡_가사(37개).csv')  # 파일 경로에 맞게 수정
lyrics = data['가사']

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(lyrics)

In [ ]:
# KMeans 클러스터링
num_clusters = 4  # 클러스터의 수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

/Users/idohyeong/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4, random_state=42)

In [ ]:
print("Cluster Centers:")
print(kmeans.cluster_centers_)

Cluster Centers:
[[0.         0.         0.         ... 0.         0.01562032 0.        ]
 [0.         0.         0.         ... 0.         0.         0.00353494]
 [0.00251692 0.         0.         ... 0.00871413 0.         0.        ]
 [0.         0.00790188 0.0042438  ... 0.         0.         0.        ]]


In [ ]:
# 각 클러스터별 대표 가사 선택 및 출력
cluster_representatives = []
for cluster_num in range(num_clusters):
    cluster_indices = [i for i, label in enumerate(kmeans.labels_) if label == cluster_num]
    cluster_lyrics = [lyrics[i] for i in cluster_indices]
    representative_lyric = " ".join(cluster_lyrics)
    cluster_representatives.append(representative_lyric)
    
    print(f"Cluster {cluster_num + 1} Representative Lyrics:")
    print(representative_lyric)
    print("\n")

Cluster 1 Representative Lyrics:
I’m such a good surfer
가라앉지 않기
비틀 비 비틀 거리다가 풍덩 uh
빠지더라도 구명복 따윈 졸업

I’m such a good surfer
휩쓸리지 않기
울렁 우 울렁 거리다가 throw up
게워내더라도 지는 건 난 못 참아

제일 높은 파도
올라타타 라차차우아
해일과 함께 사라질 타이밍
그건 내가 골라
무슨 소리 겁이 나기는, 재밌지 뭐

어어어 푸푸푸 또
허허허 우우우적
거거거 리더던 시
저저절 나라면
워어언 이 사람아
언제적 얘길 꺼내나
보란듯이 헤엄치기 처첨버벙

저저적 셔셔셔 또
저저적 셔셔셔
거거겁 없이 몸을
더더던 져져져
워어언 이 사람아
뭘 그렇게나 놀래나
더 재밌는 걸 보여줄게
When the wave has come

I’m a bubble maker
잔잔한 바다를
지긋 지 지긋 괴롭히는 villain uh
나이를 먹어도 이 버릇 남 못 줘 난

I’m a bubble maker
이제 이쯤이야
출렁 추 출렁 멀미도 suck it up
물 먹이더라도 이곳을 난 못 떠나

제일 높은 파도
올라타타 라차차우아
너울과 함께 부서질 타이밍
그건 내가 골라
무슨 소리 겁이 나기는, 재밌지 뭐

어어어 푸푸푸 또
허허허 우우우적
거거거 리더던 시
저저절 나라면
워어언 이 사람아
언제적 얘길 꺼내나
맨몸으로 헤엄치기 처첨버벙

어 또 보네 다음에
다시 만날 때까지
부디 행운을 빌어
지겹게 보자고

슬 가볼게 나 먼저
저기 물마루에
원 이 사람 참
뭘 그렇게나 놀래나
더 재밌는 걸 보여줄게
When the wave has come

저저적 셔셔셔 또
저저적 셔셔셔
거거겁 없이 몸을
더더던 져져져
워어언 이 사람아
뭘 그렇게나 보채나
거품처럼 사라질게
When the time has come 삼촌 이제 오세요
오늘도 술 좀 마신 건가요
그래도 한 손에는
조카들 좋아할 과자 들고서
너무 속상해 말아요
이리 치이고 저리 치여도
할머니 구박에도
기가 죽으면 안 돼요
난 믿어요 우리 삼

In [32]:
# 라벨 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(cluster_representatives)

# 텍스트 전처리
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(cluster_representatives)
sequences = tokenizer.texts_to_sequences(cluster_representatives)
padded_sequences = pad_sequences(sequences, maxlen=200)

# 학습 데이터와 테스트 데이터로 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [34]:
# LSTM 모델 생성
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)